# Análise e Modelagem Preditiva
Autor: Jorge Ivan Loureiro Filgueiras

Notebook completo com todas as etapas — desde a limpeza dos dados até a interpretação dos resultados.

## 1. Carregamento e limpeza dos dados

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_curve, auc, precision_recall_curve
import warnings
from google.colab import files

warnings.filterwarnings('ignore')

print("Selecione o arquivo CSV para análise:")
uploaded = files.upload()
filename = list(uploaded.keys())[0]

df = pd.read_csv(filename)
print(f"Arquivo '{filename}' carregado com sucesso!")
print("Dimensões iniciais:", df.shape)
df.info()

df.drop_duplicates(inplace=True)
df.dropna(inplace=True)
print("\nApós limpeza:", df.shape)

## 2. Estatísticas descritivas básicas

In [ ]:
df.describe().T

## 3. Histogramas / Boxplots / Gráficos de barras

In [ ]:
df.hist(figsize=(10,8), bins=20)
plt.tight_layout()
plt.show()

for col in df.select_dtypes(include='number').columns:
    plt.figure(figsize=(6,3))
    sns.boxplot(df[col], color='lightblue')
    plt.title(f'Boxplot de {col}')
    plt.show()

## 4. Gráficos de dispersão e mapa de calor

In [ ]:
num_cols = df.select_dtypes(include='number').columns

if len(num_cols) >= 2:
    sns.pairplot(df[num_cols].iloc[:, :4])
    plt.show()

time_cols = [col for col in df.columns if any(x in col.lower() for x in ['tempo', 'data', 'hora', 'dia'])]
num_cols = [col for col in num_cols if col not in time_cols]
print(f"Colunas excluídas da correlação (tempo): {time_cols}")

plt.figure(figsize=(8,6))
sns.heatmap(df[num_cols].corr(), annot=True, cmap='coolwarm')
plt.title('Mapa de calor das correlações (sem variáveis de tempo)')
plt.show()

## 5. Identificação de padrões e correlações

In [ ]:
corrs = df[num_cols].corr().unstack().sort_values(ascending=False)
print('Principais correlações:')
print(corrs[(corrs < 1)].head(5))

## 6. Interpretação textual dos insights

In [ ]:
print('As correlações identificadas indicam relações fortes entre algumas variáveis, o que pode sugerir dependência linear relevante para modelagem.')

## 7. Definição das variáveis dependente e independente

In [ ]:
y = df[num_cols[-1]]
X = df[num_cols[:-1]]
print('Variável dependente:', y.name)
print('Variáveis independentes:', list(X.columns))

## 8. Criação do modelo de regressão simples

In [ ]:
model_lr = LinearRegression()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
model_lr.fit(X_train, y_train)
y_pred = model_lr.predict(X_test)
print('Modelo de regressão linear ajustado com sucesso.')

## 9. Gráfico de dispersão com linha de tendência

In [ ]:
plt.figure(figsize=(6,4))
plt.scatter(y_test, y_pred, color='blue', alpha=0.6)
plt.plot([y.min(), y.max()], [y.min(), y.max()], 'r--')
plt.xlabel('Valores reais')
plt.ylabel('Preditos')
plt.title('Dispersão com linha de tendência (Regressão Linear)')
plt.show()

## 10. Análise de resíduos

In [ ]:
residuos = y_test - y_pred
sns.histplot(residuos, kde=True, color='orange')
plt.title('Distribuição dos resíduos')
plt.show()

print('Média dos resíduos:', np.mean(residuos))
print('Desvio padrão dos resíduos:', np.std(residuos))

## 11. Texto explicativo dos resultados da regressão

In [ ]:
print('A regressão linear apresentou uma relação proporcional entre as variáveis, com resíduos centrados próximos de zero, indicando bom ajuste.')

## 12. Treinamento de classificadores simples (Regressão Logística, Árvore, KNN)

In [ ]:
y_bin = (y > y.median()).astype(int)
X_train, X_test, y_train, y_test = train_test_split(X, y_bin, test_size=0.3, random_state=42)

models = {
    'Regressão Logística': LogisticRegression(max_iter=1000),
    'Árvore de Decisão': DecisionTreeClassifier(max_depth=4, random_state=42),
    'KNN (k=5)': KNeighborsClassifier(n_neighbors=5)
}

for name, model in models.items():
    model.fit(X_train, y_train)
print('Classificadores treinados com sucesso.')

## 13. Cálculo de acurácia, precisão, recall e F1-score

In [ ]:
for name, model in models.items():
    y_pred = model.predict(X_test)
    print(f'\n--- {name} ---')
    print('Acurácia:', accuracy_score(y_test, y_pred))
    print('Precisão:', precision_score(y_test, y_pred))
    print('Recall:', recall_score(y_test, y_pred))
    print('F1:', f1_score(y_test, y_pred))

## 14. Exibição da matriz de confusão

In [ ]:
for name, model in models.items():
    y_pred = model.predict(X_test)
    cm = confusion_matrix(y_test, y_pred)
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title(f'Matriz de Confusão - {name}')
    plt.xlabel('Previsto')
    plt.ylabel('Real')
    plt.show()

## 15. Curva ROC e Precision-Recall

In [ ]:
for name, model in models.items():
    if hasattr(model, 'predict_proba'):
        y_prob = model.predict_proba(X_test)[:,1]
        fpr, tpr, _ = roc_curve(y_test, y_prob)
        roc_auc = auc(fpr, tpr)
        plt.plot(fpr, tpr, label=f'{name} (AUC={roc_auc:.2f})')
plt.plot([0,1], [0,1], 'r--')
plt.xlabel('FPR')
plt.ylabel('TPR')
plt.title('Curvas ROC')
plt.legend()
plt.show()

for name, model in models.items():
    if hasattr(model, 'predict_proba'):
        y_prob = model.predict_proba(X_test)[:,1]
        precision, recall, _ = precision_recall_curve(y_test, y_prob)
        plt.plot(recall, precision, label=f'{name}')
plt.xlabel('Recall')
plt.ylabel('Precisão')
plt.title('Curvas Precision-Recall')
plt.legend()
plt.show()

## 16. Interpretação dos resultados de classificação

In [ ]:
Os resultados demonstram que alguns classificadores apresentaram melhor equilíbrio entre precisão e recall, dependendo do tipo de dado e separabilidade.

## 17. Síntese dos achados principais

In [ ]:
- As variáveis numéricas apresentaram correlações consistentes após remoção das temporais.
- O modelo de regressão apresentou resíduos equilibrados.
- Entre os classificadores, destacou-se aquele com melhor F1-score.
- As curvas ROC e Precision-Recall evidenciaram bom equilíbrio entre precisão e recall.